# CNN-Modèle

Avec nos données, on peut maintenant passer à la partie modélisation

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
import pandas as pd
import numpy as np

In [56]:
X = pd.read_csv('D:/Données/ENSAE/2A/S2/Séminaire de modélisation statistique/X.csv')
Y = pd.read_csv('D:/Données/ENSAE/2A/S2/Séminaire de modélisation statistique/Y.csv')
X_and_Y = pd.read_csv('D:/Données/ENSAE/2A/S2/Séminaire de modélisation statistique/X_and_Y.csv')

In [57]:
# ATTENTION, il ne faut le lancer qu'une fois A METTRE SUR L'AUTRE CODE
X['engine_serial_number'] = X['engine_serial_number'].str[4:]
X['engine_series'] = X['engine_series'].str[14:]
X['config_A'] = X['config_A'].str[9:]
X['config_B'] = X['config_B'].str[9:]
X = X.drop('Unnamed: 0', axis=1)
Y =Y.drop('Unnamed: 0', axis=1)
X_and_Y = X_and_Y.drop('Unnamed: 0', axis=1)
X = X.astype(float)

In [58]:
X

,engine_serial_number,engine_series,cycles_counter,config_A,config_B,Interpolate_flight_leg_hours,event_rank,Interpolate_SV_rank,Interpolate_Config_B_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,id_int
0,1041.0,1.0,1051.0,2.0,1.0,0.811111,2.0,0.0,0.0,-0.413768,-0.337710,-0.498373,0.0,-1.760685,499.0
1,1041.0,1.0,1067.0,2.0,1.0,2.771667,2.0,0.0,0.0,-0.487399,-0.368209,-0.919261,0.0,0.400740,499.0
2,1041.0,1.0,1026.0,2.0,1.0,2.071667,2.0,0.0,0.0,-0.390356,-0.367936,0.027738,0.0,0.199678,499.0
3,1041.0,1.0,1044.0,2.0,1.0,3.485000,2.0,0.0,0.0,-0.479503,-0.337710,-0.919261,0.0,-1.660153,499.0
4,1041.0,1.0,1041.0,2.0,1.0,1.550556,2.0,0.0,0.0,-0.580330,-0.356625,-1.234928,0.0,-2.665467,499.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585095,1.0,1.0,1403.0,1.0,1.0,4.098889,2.0,0.0,0.0,-0.251725,4.245779,-0.393150,0.0,-0.956433,6349.0
585096,1.0,1.0,1428.0,1.0,1.0,0.727778,2.0,0.0,0.0,-0.574341,0.531657,-1.234928,0.0,0.501272,6349.0
585097,1.0,1.0,267.0,1.0,1.0,0.727222,0.0,0.0,0.0,-0.475763,0.193871,0.448627,0.0,-0.101917,6349.0
585098,1.0,1.0,2183.0,1.0,2.0,1.103889,5.0,0.0,3.0,-0.513067,0.017431,1.079960,0.0,-1.408825,6349.0


In [59]:
Y

,Interpolate_egt_slope,id_int
0,-0.024441,499
1,-0.024441,499
2,-0.024441,499
3,-0.024441,499
4,-0.024441,499
...,...,...
585095,0.002829,6349
585096,0.002829,6349
585097,-0.029193,6349
585098,-0.398132,6349


In [60]:
### On garde 75% des données pour le train et 25% pour le test

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(438825, 15) (146275, 15) (438825, 2) (146275, 2)


### Multiple Input Series

In [61]:
# multivariate cnn example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [62]:
X.columns

Index(['engine_serial_number', 'engine_series', 'cycles_counter', 'config_A',
       'config_B', 'Interpolate_flight_leg_hours', 'event_rank',
       'Interpolate_SV_rank', 'Interpolate_Config_B_rank',
       'Interpolate_var_env_1', 'Interpolate_var_env_2',
       'Interpolate_var_env_3', 'Interpolate_var_env_4',
       'Interpolate_var_env_5', 'id_int'],
      dtype='object')

Les deux cellules ci-dessous, je vais le refaire au propre après

In [63]:
# On réalise l'input train
X_train_engine_serial_number = X_train['engine_serial_number']
X_train_engine_series = X_train['engine_series']
X_train_cycles_counter = X_train['cycles_counter']
X_train_config_A = X_train['config_A']
X_train_config_B = X_train['config_B']
X_train_Interpolate_flight_leg_hours = X_train['Interpolate_flight_leg_hours']
X_train_event_rank = X_train['event_rank']
X_train_Interpolate_SV_rank = X_train['Interpolate_SV_rank']
X_train_Interpolate_Config_B_rank  = X_train['Interpolate_Config_B_rank']
X_train_Interpolate_var_env_1 = X_train['Interpolate_var_env_1']
X_train_Interpolate_var_env_2 = X_train['Interpolate_var_env_2']
X_train_Interpolate_var_env_3 = X_train['Interpolate_var_env_3']
X_train_Interpolate_var_env_4 = X_train['Interpolate_var_env_4']
X_train_Interpolate_var_env_5 = X_train['Interpolate_var_env_5'] 

#Et l'output train
Y_train_Interpolate_egt_slope = Y_train['Interpolate_egt_slope'] 

# On définit l'input test
in_seq1_train = array(X_train_engine_serial_number)
in_seq2_train = array(X_train_engine_series)
in_seq3_train = array(X_train_cycles_counter)
in_seq4_train = array(X_train_config_A)
in_seq5_train = array(X_train_config_B)
in_seq6_train = array(X_train_Interpolate_flight_leg_hours)
in_seq7_train = array(X_train_event_rank)
in_seq8_train = array(X_train_Interpolate_SV_rank)
in_seq9_train = array(X_train_Interpolate_Config_B_rank)
in_seq10_train = array(X_train_Interpolate_var_env_1)
in_seq11_train = array(X_train_Interpolate_var_env_2)
in_seq12_train = array(X_train_Interpolate_var_env_3)
in_seq13_train = array(X_train_Interpolate_var_env_4)
in_seq14_train = array(X_train_Interpolate_var_env_5)

# On définit l'output test
out_seq_train = np.array(Y_train_Interpolate_egt_slope)

#On convertit en une structure [ligne, colonne]
in_seq1_train = in_seq1_train.reshape((len(in_seq1_train), 1))
in_seq2_train = in_seq2_train.reshape((len(in_seq2_train), 1))
in_seq3_train = in_seq3_train.reshape((len(in_seq3_train), 1))
in_seq4_train = in_seq4_train.reshape((len(in_seq4_train), 1))
in_seq5_train = in_seq5_train.reshape((len(in_seq5_train), 1))
in_seq6_train = in_seq6_train.reshape((len(in_seq6_train), 1))
in_seq7_train = in_seq7_train.reshape((len(in_seq7_train), 1))
in_seq8_train = in_seq8_train.reshape((len(in_seq8_train), 1))
in_seq9_train = in_seq9_train.reshape((len(in_seq9_train), 1))
in_seq10_train = in_seq10_train.reshape((len(in_seq10_train), 1))
in_seq11_train = in_seq11_train.reshape((len(in_seq11_train), 1))
in_seq12_train = in_seq12_train.reshape((len(in_seq12_train), 1))
in_seq13_train = in_seq13_train.reshape((len(in_seq13_train), 1))
in_seq14_train = in_seq14_train.reshape((len(in_seq14_train), 1))

out_seq_train = out_seq_train.reshape((len(out_seq_train), 1))

In [64]:
dataset_train = hstack((in_seq1_train, in_seq2_train, in_seq3_train, 
                        in_seq4_train, in_seq5_train, in_seq6_train,
                        in_seq7_train, in_seq8_train, in_seq9_train, 
                        in_seq10_train, in_seq11_train, in_seq12_train,
                        in_seq13_train, in_seq14_train, out_seq_train))
print(dataset_train)

[[ 3.11000000e+02  1.00000000e+00  3.57700000e+03 ...  0.00000000e+00
   4.88805700e-02 -3.82366600e-02]
 [ 3.74000000e+02  1.00000000e+00  3.63700000e+03 ...  0.00000000e+00
   1.17218317e+00 -5.08319000e-02]
 [ 6.58000000e+02  3.00000000e+00  8.43000000e+02 ...  0.00000000e+00
  -1.52182200e-01 -6.54409600e-02]
 ...
 [ 2.94000000e+02  1.00000000e+00  9.65000000e+02 ...  0.00000000e+00
   5.01271900e-01 -3.53901300e-02]
 [ 4.81000000e+02  3.00000000e+00  2.37600000e+03 ...  0.00000000e+00
  -5.54307800e-01 -9.49736800e-03]
 [ 2.11000000e+02  1.00000000e+00  1.01400000e+03 ...  1.00000000e+00
   1.20499200e+00 -4.81398800e-02]]


In [65]:
#On réalise l'input test
X_test_engine_serial_number = X_test['engine_serial_number']
X_test_engine_series = X_test['engine_series']
X_test_cycles_counter = X_test['cycles_counter']
X_test_config_A = X_test['config_A']
X_test_config_B = X_test['config_B']
X_test_Interpolate_flight_leg_hours = X_test['Interpolate_flight_leg_hours']
X_test_event_rank = X_test['event_rank']
X_test_Interpolate_SV_rank = X_test['Interpolate_SV_rank']
X_test_Interpolate_Config_B_rank  = X_test['Interpolate_Config_B_rank']
X_test_Interpolate_var_env_1 = X_test['Interpolate_var_env_1']
X_test_Interpolate_var_env_2 = X_test['Interpolate_var_env_2']
X_test_Interpolate_var_env_3 = X_test['Interpolate_var_env_3']
X_test_Interpolate_var_env_4 = X_test['Interpolate_var_env_4']
X_test_Interpolate_var_env_5 = X_test['Interpolate_var_env_5'] 

#Et l'output train
Y_test_Interpolate_egt_slope = Y_test['Interpolate_egt_slope']

# On définit l'input test
in_seq1_test = array(X_test_engine_serial_number)
in_seq2_test = array(X_test_engine_series)
in_seq3_test = array(X_test_cycles_counter)
in_seq4_test = array(X_test_config_A)
in_seq5_test = array(X_test_config_B)
in_seq6_test = array(X_test_Interpolate_flight_leg_hours)
in_seq7_test = array(X_test_event_rank)
in_seq8_test = array(X_test_Interpolate_SV_rank)
in_seq9_test = array(X_test_Interpolate_Config_B_rank)
in_seq10_test = array(X_test_Interpolate_var_env_1)
in_seq11_test = array(X_test_Interpolate_var_env_2)
in_seq12_test = array(X_test_Interpolate_var_env_3)
in_seq13_test = array(X_test_Interpolate_var_env_4)
in_seq14_test = array(X_test_Interpolate_var_env_5)

# On définit l'output test
out_seq_test = np.array(Y_test_Interpolate_egt_slope)

#On convertit en une structure [ligne, colonne]
in_seq1_test = in_seq1_test.reshape((len(in_seq1_test), 1))
in_seq2_test = in_seq2_test.reshape((len(in_seq2_test), 1))
in_seq3_test = in_seq3_test.reshape((len(in_seq3_test), 1))
in_seq4_test = in_seq4_test.reshape((len(in_seq4_test), 1))
in_seq5_test = in_seq5_test.reshape((len(in_seq5_test), 1))
in_seq6_test = in_seq6_test.reshape((len(in_seq6_test), 1))
in_seq7_test = in_seq7_test.reshape((len(in_seq7_test), 1))
in_seq8_test = in_seq8_test.reshape((len(in_seq8_test), 1))
in_seq9_test = in_seq9_test.reshape((len(in_seq9_test), 1))
in_seq10_test = in_seq10_test.reshape((len(in_seq10_test), 1))
in_seq11_test = in_seq11_test.reshape((len(in_seq11_test), 1))
in_seq12_test = in_seq12_test.reshape((len(in_seq12_test), 1))
in_seq13_test = in_seq13_test.reshape((len(in_seq13_test), 1))
in_seq14_test = in_seq14_test.reshape((len(in_seq14_test), 1))

out_seq_test = out_seq_test.reshape((len(out_seq_test), 1))

In [66]:
dataset_test = hstack((in_seq1_test, in_seq2_test, in_seq3_test, 
                       in_seq4_test, in_seq5_test, in_seq6_test,
                       in_seq7_test, in_seq8_test, in_seq9_test,
                       in_seq10_test, in_seq11_test, in_seq12_test,
                       in_seq13_test, in_seq14_test, out_seq_test))
print(dataset_test)

[[ 1.120000e+02  1.000000e+00  1.593000e+03 ...  0.000000e+00
  -2.363873e+00 -7.463315e-03]
 [ 4.710000e+02  1.000000e+00  1.728000e+03 ...  0.000000e+00
   6.018033e-01 -1.816048e-02]
 [ 8.900000e+02  1.000000e+00  8.770000e+02 ...  0.000000e+00
   4.007405e-01 -3.095684e-02]
 ...
 [ 6.700000e+02  1.000000e+00  4.190000e+02 ...  0.000000e+00
  -1.207762e+00 -1.495423e-02]
 [ 4.190000e+02  1.000000e+00  2.335000e+03 ...  0.000000e+00
  -1.006699e+00  8.755395e-03]
 [ 3.510000e+02  1.000000e+00  2.595000e+03 ...  0.000000e+00
   1.204992e+00 -3.502007e-01]]


In [67]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [68]:
n_steps = 100
# convert into input/output
#ATTENTION cette cellule peut prendre plusieurs minutes à s'éxécuter
X_train, y_train = split_sequences(dataset_train, n_steps)


In [69]:
X_test, y_test = split_sequences(dataset_test, n_steps)

In [70]:
# the dataset knows the number of features, e.g. 2
n_features = X_train.shape[2] # On a 14 colonnes d'input

In [71]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [72]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 100, 14) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
conv1d_2 (None, 100, 14) float32
max_pooling1d_2 (None, 99, 64) float32
flatten_2 (None, 49, 64) float32
dense_4 (None, 3136) float32
dense_5 (None, 50) float32


[None, None, None, None, None]

In [73]:
# On fit le modèle, et ça prend vachement longtemps! ENVIRON 1H
model.fit(X_train, y_train, epochs=10, verbose=0)

In [74]:
#on prédit des valeurs pour y
yhat = model.predict(X_test, verbose=0)
print(yhat)

[[-0.03669192]
 [-0.03669192]
 [-0.03669192]
 ...
 [-0.03669192]
 [-0.03669192]
 [-0.03669192]]


In [75]:
yhat

array([[-0.03669192],
       [-0.03669192],
       [-0.03669192],
       ...,
       [-0.03669192],
       [-0.03669192],
       [-0.03669192]], dtype=float32)

Maintenant, il faudrait compter le nombre de bonnes prédictions qu'on a eu, pour "noter notre modèle"